##### Lets research strategies and aggregate their results

In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ta
import plotly_express as px
import plotly.graph_objects as go
import yfinance as yf
%matplotlib inline

In [4]:
# Read in stock data
stock = 'CBD'
data = yf.download(stock)
print(data.shape)
data.tail()

[*********************100%***********************]  1 of 1 completed
(6094, 6)


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-05-18,9.71,9.82,9.46,9.80,9.80,991100
2020-05-19,9.82,9.83,9.50,9.50,9.50,677200
2020-05-20,9.79,9.93,9.67,9.71,9.71,399100
2020-05-21,9.81,10.10,9.81,10.05,10.05,602300
2020-05-22,10.46,10.78,10.19,10.28,10.28,744000


In [5]:
# Calculate technical indicators
data = ta.add_all_ta_features(data,open='Open',high='High',low='Low',close='Close',volume='Volume',fillna=True)
print(data.shape)
data.head()

/Users/HarrisonHoffman/opt/anaconda3/lib/python3.7/site-packages/ta/trend.py:543: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (self._dip[i]/self._trs[i])
/Users/HarrisonHoffman/opt/anaconda3/lib/python3.7/site-packages/ta/trend.py:547: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (self._din[i]/self._trs[i])


(6094, 73)


,Open,High,Low,Close,Adj Close,Volume,volume_adi,volume_obv,volume_cmf,volume_fi,...,momentum_uo,momentum_stoch,momentum_stoch_signal,momentum_wr,momentum_ao,momentum_kama,momentum_roc,others_dr,others_dlr,others_cr
Date,,,,,,,,,,,,,,,,,,,,,
1996-03-08,6.5000,6.6875,6.5000,6.6875,3.573622,54000,54000.0,54000,1.000000,0.000000,...,50.0,100.000000,100.000000,-0.000000,0.0,6.6875,0.0,-67.615330,0.000000,0.000000
1996-03-11,6.8750,6.8750,6.8750,6.8750,3.673817,400,54000.0,54400,0.992647,75.000000,...,50.0,100.000000,100.000000,-0.000000,0.0,6.8750,0.0,2.803738,2.765153,2.803738
1996-03-12,6.8750,6.8750,6.8750,6.8750,3.673817,0,54000.0,54400,0.992647,64.285714,...,50.0,100.000000,100.000000,-0.000000,0.0,6.8750,0.0,0.000000,0.000000,2.803738
1996-03-13,6.8125,6.8125,6.8125,6.8125,3.640418,10000,54000.0,44400,0.838509,-34.183673,...,50.0,83.333333,94.444444,-16.666667,0.0,6.8125,0.0,-0.909091,-0.913248,1.869159
1996-03-14,6.8750,6.8750,6.8750,6.8750,3.673817,10200,54000.0,54600,0.723861,61.771137,...,50.0,100.000000,94.444444,-0.000000,0.0,6.8750,0.0,0.917431,0.913248,2.803738


In [6]:
data.columns

Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'volume_adi',
       'volume_obv', 'volume_cmf', 'volume_fi', 'volume_em', 'volume_sma_em',
       'volume_vpt', 'volume_nvi', 'volatility_atr', 'volatility_bbm',
       'volatility_bbh', 'volatility_bbl', 'volatility_bbw', 'volatility_bbhi',
       'volatility_bbli', 'volatility_kcc', 'volatility_kch', 'volatility_kcl',
       'volatility_kchi', 'volatility_kcli', 'volatility_dcl',
       'volatility_dch', 'volatility_dchi', 'volatility_dcli', 'trend_macd',
       'trend_macd_signal', 'trend_macd_diff', 'trend_ema_fast',
       'trend_ema_slow', 'trend_adx', 'trend_adx_pos', 'trend_adx_neg',
       'trend_vortex_ind_pos', 'trend_vortex_ind_neg', 'trend_vortex_ind_diff',
       'trend_trix', 'trend_mass_index', 'trend_cci', 'trend_dpo', 'trend_kst',
       'trend_kst_sig', 'trend_kst_diff', 'trend_ichimoku_a',
       'trend_ichimoku_b', 'trend_visual_ichimoku_a',
       'trend_visual_ichimoku_b', 'trend_aroon_up', 'trend_aro

### RSI Strategy
If RSI crosses above 30: buy. If RSI crosses below 70: sell

### CCI Strategy
If CCI crosses above -100: buy. If CCI crosses below 100: sell

In [13]:
def recommender(data):
    recommendations = []
    # RSI
    rsi = data['momentum_rsi']
    if rsi[-1] > 30 and rsi[-3] < 30:
        recommendations.append(1)
    elif rsi[-1] < 70  and rsi[-3] > 70:
        recommendations.append(0)
    else:
        recommendations.append(.5)
    # CCI
    cci = data['trend_cci']
    if (cci[-1] > -100 and cci[-1] < -50) and cci[-5] < -100:
        recommendations.append(1)
    elif cci[-1] < 100 and cci[-5] > 100:
          recommendations.append(0)
    else:
          recommendations.append(.5)
          
    return np.mean(recommendations)

In [14]:
recommender(data)

0.75